In [1]:
import os
os.chdir("/Users/jacksonwalters/Documents/GitHub/enefit-kaggle/predict-energy-behavior-of-prosumers/")
!ls

client.csv                            gas_prices.csv
county_id_to_name_map.json            historical_weather.csv
electricity_prices.csv                public_timeseries_testing_util.py
enefit                                train.csv
example_test_files                    weather_station_to_county_mapping.csv
forecast_weather.csv


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
#helper function to convert datetime strings to integers representing a time year-month-day hour-min-sec
from datetime import datetime
def datestr_to_int(datetime_str,date_format):
    return datetime.strptime(datetime_str, date_format).timestamp()

In [4]:
#load the training data, dropping NaN's
train = pd.read_csv("train.csv").dropna()
train['datetime'] = train['datetime'].apply(lambda x: datestr_to_int(x,'%Y-%m-%d %H:%M:%S'))

In [6]:
forecast_weather = pd.read_csv("forecast_weather.csv")
#convert strings to ints
forecast_weather['origin_datetime'] = forecast_weather['origin_datetime'].apply(lambda x: datestr_to_int(x,'%Y-%m-%d %H:%M:%S%z'))
forecast_weather['forecast_datetime'] = forecast_weather['forecast_datetime'].apply(lambda x: datestr_to_int(x,'%Y-%m-%d %H:%M:%S%z'))

In [7]:
weather_station_to_county_mapping = pd.read_csv("weather_station_to_county_mapping.csv")

In [8]:
historical_weather = pd.read_csv("historical_weather.csv")

In [9]:
gas_prices = pd.read_csv("gas_prices.csv")
#convert date strings to ints
gas_prices['forecast_date'] = gas_prices['forecast_date'].apply(lambda x: datestr_to_int(x,'%Y-%m-%d'))
gas_prices['origin_date'] = gas_prices['origin_date'].apply(lambda x: datestr_to_int(x,'%Y-%m-%d'))

In [10]:
electricity_prices = pd.read_csv("electricity_prices.csv")
#convert date strings to ints
electricity_prices['forecast_date'] = electricity_prices['forecast_date'].apply(lambda x: datestr_to_int(x,'%Y-%m-%d %H:%M:%S'))
electricity_prices['origin_date'] = electricity_prices['origin_date'].apply(lambda x: datestr_to_int(x,'%Y-%m-%d %H:%M:%S'))

In [11]:
forecast_weather['origin_datetime'] - forecast_weather['forecast_datetime']

0           -3600.0
1           -3600.0
2           -3600.0
3           -3600.0
4           -3600.0
             ...   
3424507   -172800.0
3424508   -172800.0
3424509   -172800.0
3424510   -172800.0
3424511   -172800.0
Length: 3424512, dtype: float64

In [90]:
gas_prices

,forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id
0,1.630469e+09,45.23,46.32,1.630382e+09,1
1,1.630555e+09,45.62,46.29,1.630469e+09,2
2,1.630642e+09,45.85,46.40,1.630555e+09,3
3,1.630728e+09,46.30,46.80,1.630642e+09,4
4,1.630814e+09,46.30,46.58,1.630728e+09,5
...,...,...,...,...,...
632,1.685074e+09,29.10,34.10,1.684987e+09,633
633,1.685160e+09,28.30,34.10,1.685074e+09,634
634,1.685246e+09,28.10,34.10,1.685160e+09,635
635,1.685333e+09,28.16,36.98,1.685246e+09,636


In [120]:
electricity_prices

,forecast_date,euros_per_mwh,origin_date,data_block_id
0,1.630469e+09,92.51,1.630382e+09,1
1,1.630472e+09,88.90,1.630386e+09,1
2,1.630476e+09,87.35,1.630390e+09,1
3,1.630480e+09,86.88,1.630393e+09,1
4,1.630483e+09,88.43,1.630397e+09,1
...,...,...,...,...
15281,1.685488e+09,82.10,1.685401e+09,637
15282,1.685491e+09,150.85,1.685405e+09,637
15283,1.685495e+09,82.10,1.685408e+09,637
15284,1.685498e+09,82.09,1.685412e+09,637


In [126]:
#merge gas prices and train.csv data
#column names differ, so use left_on and right_on
df = pd.merge(train, gas_prices, left_on='datetime', right_on='origin_date', how='outer')

In [127]:
df = df.drop('data_block_id_x',axis=1).drop('data_block_id_y',axis=1)

In [128]:
df.dropna()

,county,is_business,product_type,target,is_consumption,datetime,row_id,prediction_unit_id,forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date
0,0.0,0.0,1.0,0.713,0.0,1.630469e+09,0.0,0.0,1.630555e+09,45.62,46.29,1.630469e+09
1,0.0,0.0,1.0,96.590,1.0,1.630469e+09,1.0,0.0,1.630555e+09,45.62,46.29,1.630469e+09
2,0.0,0.0,2.0,0.000,0.0,1.630469e+09,2.0,1.0,1.630555e+09,45.62,46.29,1.630469e+09
3,0.0,0.0,2.0,17.314,1.0,1.630469e+09,3.0,1.0,1.630555e+09,45.62,46.29,1.630469e+09
4,0.0,0.0,3.0,2.904,0.0,1.630469e+09,4.0,2.0,1.630555e+09,45.62,46.29,1.630469e+09
...,...,...,...,...,...,...,...,...,...,...,...,...
2008589,15.0,1.0,0.0,163.077,1.0,1.685333e+09,2009117.0,64.0,1.685419e+09,29.00,34.00,1.685333e+09
2008590,15.0,1.0,1.0,0.000,0.0,1.685333e+09,2009118.0,59.0,1.685419e+09,29.00,34.00,1.685333e+09
2008591,15.0,1.0,1.0,33.713,1.0,1.685333e+09,2009119.0,59.0,1.685419e+09,29.00,34.00,1.685333e+09
2008592,15.0,1.0,3.0,0.000,0.0,1.685333e+09,2009120.0,60.0,1.685419e+09,29.00,34.00,1.685333e+09


In [129]:
#merge train and gas_prices via left join on data_block_id
#this leaves all rows of train, but matches 
df = df.merge(electricity_prices, on=['origin_date','forecast_date'], how='outer')

In [130]:
df.dropna()

,county,is_business,product_type,target,is_consumption,datetime,row_id,prediction_unit_id,forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,euros_per_mwh,data_block_id
0,0.0,0.0,1.0,0.713,0.0,1.630469e+09,0.0,0.0,1.630555e+09,45.62,46.29,1.630469e+09,96.99,2.0
1,0.0,0.0,1.0,96.590,1.0,1.630469e+09,1.0,0.0,1.630555e+09,45.62,46.29,1.630469e+09,96.99,2.0
2,0.0,0.0,2.0,0.000,0.0,1.630469e+09,2.0,1.0,1.630555e+09,45.62,46.29,1.630469e+09,96.99,2.0
3,0.0,0.0,2.0,17.314,1.0,1.630469e+09,3.0,1.0,1.630555e+09,45.62,46.29,1.630469e+09,96.99,2.0
4,0.0,0.0,3.0,2.904,0.0,1.630469e+09,4.0,2.0,1.630555e+09,45.62,46.29,1.630469e+09,96.99,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017819,15.0,1.0,0.0,163.077,1.0,1.685333e+09,2009117.0,64.0,1.685419e+09,29.00,34.00,1.685333e+09,8.57,637.0
2017820,15.0,1.0,1.0,0.000,0.0,1.685333e+09,2009118.0,59.0,1.685419e+09,29.00,34.00,1.685333e+09,8.57,637.0
2017821,15.0,1.0,1.0,33.713,1.0,1.685333e+09,2009119.0,59.0,1.685419e+09,29.00,34.00,1.685333e+09,8.57,637.0
2017822,15.0,1.0,3.0,0.000,0.0,1.685333e+09,2009120.0,60.0,1.685419e+09,29.00,34.00,1.685333e+09,8.57,637.0


In [131]:
df = df.merge(forecast_weather, left_on='forecast_date', right_on='forecast_datetime',how='outer')

In [132]:
#drop NaN rows
df

,county,is_business,product_type,target,is_consumption,datetime,row_id,prediction_unit_id,forecast_date,lowest_price_per_mwh,...,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,data_block_id_y,forecast_datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
0,0.0,0.0,1.0,0.713,0.0,1.630469e+09,0.0,0.0,1.630555e+09,45.62,...,0.647781,0.695312,0.886121,-9.359729,1.0,1.630555e+09,97.381389,40.216389,0.0,0.000014
1,0.0,0.0,1.0,0.713,0.0,1.630469e+09,0.0,0.0,1.630555e+09,45.62,...,0.653580,0.703644,1.224378,-4.953479,1.0,1.630555e+09,139.194722,47.505278,0.0,0.000000
2,0.0,0.0,1.0,0.713,0.0,1.630469e+09,0.0,0.0,1.630555e+09,45.62,...,0.326401,0.400497,1.668348,-7.180285,1.0,1.630555e+09,220.261389,61.016389,0.0,0.000004
3,0.0,0.0,1.0,0.713,0.0,1.630469e+09,0.0,0.0,1.630555e+09,45.62,...,0.112305,0.159805,1.965345,-9.117541,1.0,1.630555e+09,279.141389,70.936389,0.0,0.000008
4,0.0,0.0,1.0,0.713,0.0,1.630469e+09,0.0,0.0,1.630555e+09,45.62,...,0.142334,0.180725,1.548841,-9.581653,1.0,1.630555e+09,290.092500,73.887500,0.0,0.000018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23996189,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.004059,0.308594,6.890626,-3.258144,637.0,1.685574e+09,0.000000,0.000000,0.0,0.000000
23996190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.198700,0.906036,6.890626,-3.768032,637.0,1.685574e+09,0.000000,0.000000,0.0,0.000000
23996191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.448853,0.530518,6.214112,-4.700405,637.0,1.685574e+09,0.000000,0.000000,0.0,0.000000
23996192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.997314,0.997498,5.810060,-4.338345,637.0,1.685574e+09,0.000000,0.000000,0.0,0.000000


In [98]:
#drop the target column to get the training data
X = df.drop("target", axis=1)

In [99]:
#set the target variable
y = df['target']

In [100]:
#verify there are no NaN's
np.any(np.isnan(y))

False

In [101]:
# load the modeling modules from sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn.metrics import mean_absolute_error

In [102]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [103]:
#create a multivariable linear regression
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(X, y)

LinearRegression()

In [104]:
#predict on the test data
lin_pred = regr.predict(X_test)

In [105]:
mean_absolute_error(lin_pred,y_test)

283.13101389852756

In [106]:
#train the random forest regressor
rf = RandomForestRegressor(n_estimators=10)
rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10)

In [107]:
#predict the target values using the random forest regressor
rf_pred = rf.predict(X_test)

In [108]:
#mean absolute error for random regressor
mean_absolute_error(y_test,rf_pred)

19.109998741650767